In [1]:
import numpy as np

row = np.array([0, 0, 0, 1, 1, 1, 2, 2, 2])  # Example row indices

grad_row = np.stack([row * 2, row * 2 + 1], axis=1).flatten()
print(grad_row)


[0 1 0 1 0 1 2 3 2 3 2 3 4 5 4 5 4 5]


In [6]:
import numpy as np
from pcdiff import knn_graph, estimate_basis, build_grad_div, laplacian, coords_projected, weighted_least_squares, gaussian_weights
import numpy as np
import numpy.linalg as LA
from scipy.sparse import coo_matrix

regularizer=1e-8
shape_regularizer=None

data = np.loadtxt('C:/Users/aagaa/Documents/GitHub/R-D/Code/Leihui Code/src/data/bunny.xyz')
pos = data[:, 0:3]
# Random point cloud
#pos = np.random.rand(1000, 3)

Cloud_size = np.size(pos,0)

# Generate kNN graph
edge_index = knn_graph(pos, 20)
# Estimate normals and local frames
basis = estimate_basis(pos, edge_index)

row, col = edge_index
k = (row == 0).sum()

# Get coordinates in tangent plane by projecting along the normal of the plane
coords = coords_projected(pos, *basis, edge_index)

# Compute weights based on distance in euclidean space
dist = LA.norm(pos[col] - pos[row], axis=1)
weights = gaussian_weights(dist, k, 1)

# Get weighted least squares result
# wls multiplied with a function f at k neighbors will give the coefficients c0-c5
# for the surface f(x, y) = [x, y, c0 + c1*x + c2*y + c3*x**2 + c4*xy + c5*y**2]
# defined on a neighborhood of each point.
if shape_regularizer is None:
    wls = weighted_least_squares(coords, weights, k, regularizer)
else:
    wls, wls_shape = weighted_least_squares(coords, weights, k, regularizer, shape_regularizer)

# Format as sparse matrix

# The gradient of f at (0, 0) will be
# df/dx|(0, 0) = [1, 0, c1 + 2*c3*0 + c4*0] = [1, 0, c1]
# df/dy|(0, 0) = [0, 1, c2 + c4*0 + 2*c5*0] = [0, 1, c2]
# Hence, we can use the row in wls that outputs c1 and c2 for the gradient
# in x direction and y direction, respectively
grad_row = np.stack([row * 2, row * 2 + 1], axis=1).flatten()
grad_col = np.stack([col]*2, axis=1).flatten()
grad_values = np.stack([wls[:, 1], wls[:, 2]], axis=1).flatten()

# Create gradient matrix
grad = coo_matrix((grad_values, (grad_row, grad_col)), shape=(pos.shape[0] * 2, pos.shape[0]))

print(wls.shape)
print(grad.shape)
print(pos.shape)
print(grad_row[0:40])
print(grad_col.shape)
print(grad_values.shape)
print(np.sum(grad.row==0))

(718940, 6)
(71894, 35947)
(35947, 3)
[0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0
 1 0 1]
(1437880,)
(1437880,)
20


In [9]:
from pcdiff import knn_graph, estimate_basis, build_grad_div




def calculate_gradients(pointcloud, k_neighbors):
    #TODO: Find out if this is only grad_x or just the gradient found here, maybe need more info for complete gradient
    edge_index = knn_graph(pointcloud, k_neighbors)
    normal, x_basis, y_basis = estimate_basis(pointcloud, edge_index)
    row, col = edge_index
    k = (row == 0).sum()

    # Get coordinates in tangent plane by projecting along the normal of the plane
    coords = coords_projected(pos, *basis, edge_index)

    # Compute weights based on distance in euclidean space
    dist = LA.norm(pos[col] - pos[row], axis=1)
    weights = gaussian_weights(dist, k, 1)

    # Get weighted least squares result
    # wls multiplied with a function f at k neighbors will give the coefficients c0-c5
    # for the surface f(x, y) = [x, y, c0 + c1*x + c2*y + c3*x**2 + c4*xy + c5*y**2]
    # defined on a neighborhood of each point.
    if shape_regularizer is None:
        wls = weighted_least_squares(coords, weights, k, regularizer)
    else:
        wls, wls_shape = weighted_least_squares(coords, weights, k, regularizer, shape_regularizer)
    
    gradients = wls[::20,1:3]    
    return gradients 


In [10]:
calculate_gradients(pos, 20)

array([[  3.50776946,  -1.14131284],
       [ -2.29210178,   1.41623772],
       [ -5.53423966,   4.80631342],
       ...,
       [-77.71204982, -21.36010264],
       [ 44.63317144, -15.24749962],
       [  9.5745682 ,   3.04100186]])

In [5]:

# Generate a base sequence from 1 to 300 (300 rows)
base = np.arange(1, 301).reshape(-1, 1)  # Shape: (300, 1)

# Create the 300x6 matrix with shifted values in columns
arr = base + np.arange(6)  # Broadcasting shifts each column

print(arr)

print(arr[::20,1:3])

[[  1   2   3   4   5   6]
 [  2   3   4   5   6   7]
 [  3   4   5   6   7   8]
 ...
 [298 299 300 301 302 303]
 [299 300 301 302 303 304]
 [300 301 302 303 304 305]]
[[  2   3]
 [ 22  23]
 [ 42  43]
 [ 62  63]
 [ 82  83]
 [102 103]
 [122 123]
 [142 143]
 [162 163]
 [182 183]
 [202 203]
 [222 223]
 [242 243]
 [262 263]
 [282 283]]
